### Load all libraries

In [ ]:
from utils import * 

### Relevant functions

In [ ]:
def min_div(a, m):
    """
    Find q = argmin_{q in N} |q*a - m|. This q is given by either ceil(m/a) or floor(m/a), depending on the rest.
    """
    if (a <= 0) or (m < a):
        return None
    tmp_d = {
        floor(m/a): m - a * floor(m/a),
        ceil(m/a): a * ceil(m/a) - m
    }
    return min(tmp_d, key=tmp_d.get)

# Resample stocks array
grouping_stocks = lambda s: np.vectorize(min_div)(s, max(s))

def generate_values(stock_tickers, start_date, end_date, q):
    """
    Given a list of stocks, generates the Yahoo Finance prices DataFrame prices_df. Every price series is a stored in a column. The prices df is resampled on month for the time being, and has the latest monthly datapoint as value.
    :param stock_tickers: list, list of stock names. 
    :start_date: datetime.date or datetime.datetime 
    :end_date: datetime.date or datetime.datetime
    :return: prices_df
    """
    data = YahooDataProvider(tickers=stock_tickers,
                 start = start_date,
                 end = end_date)
    data.run()

    prices = np.array([s.iloc[-1] for s in data._data])
    return prices, data.get_period_return_mean_vector(), q*data.get_period_return_covariance_matrix()

def qcmodel(prices, k, budget, mu, Q):
    """
    Quantum model. It returns the best stocks allocation and the results dictionary. 
    The whole model is computed inside the function.
    :param prices: np.array, not resampled prices 
    :param k: float, scaling factor 
    :param budget: float, budget
    :param mu: array, expected returns 
    :param Q: q*sigma, where sigma is the returns' covariance 
    :return: best allocation (for non-resampled stocks), result dictionary
    """
    # resample the prices
    grouping = grouping_stocks(prices)
    s = grouping * prices
    budget_bits = floor(np.log2(budget/np.min(s)))
    
    # build the model
    mdl = Model('portfolio_optimization')
    x = mdl.integer_var_list((f'x{i}' for i in range(len(prices))), lb=0, ub=2**budget_bits-1)

    objective = mdl.sum([mu[i]*x[i] for i in range(len(prices))])
    objective -= mdl.sum([Q[i,j]*x[i]*x[j] for i in range(len(prices)) for j in range(len(prices))])
    mdl.maximize(objective)

    norm = s.mean()/k
    mdl.add_constraint(mdl.sum(x[i] * ceil(s[i]/norm) for i in range(len(prices))) <= floor(budget/norm))
    
    return mdl, grouping 

### Insert main values for the model

In [ ]:
# main values
# stock_tickers = ['FXD', 'FXR', 'FXL', 'FTXR', 'QTEC']
stock_tickers = ['FXD', 'FXR']
start_date = date(2017, 1, 1)
computation_date = date(2020, 1, 1)
end_date = date(2021, 1, 1)
budget = 400
q = 1.
k = 1.
# max_qbits = 27

#hyperparameters
optim_dict = {
      "quantum_instance": 'qasm_simulator',
      "shots": 1024,
      "print": True,
      "logfile": True,
      "solver": 'vqe',
      "optimizer": SLSQP,
      "maxiter": 1000,
      "depth": 1,
      "alpha": 0.35
    }

### Run optimisation scheme 

In [ ]:
# building the etf
etf = {}
date_ = copy(computation_date)

while date_ < end_date: 
    # generate main values for the model
    prices, mu, Q = generate_values(stock_tickers = stock_tickers, start_date = start_date, 
                                    end_date = date_, q = 1.)
    
    # choose the optimal k, given a threshold on the max number of qbits.
    # uncomment the strings below and the max_qbits assignation above if you want k to be chosen depending on the qbits number 
#    qbits_dict = {k0: max(l0) for k0, l0 in dict_inverse({j: model_qbits(prices, j, B) for j in range(1, 100)}).items()}
#    if max_qbits in qbits_dict: 
#        k = qbits_dict[max_qbits]
#    else: 
#        warn(f'Number of qbits given not found among possible models.\nThe minimum number of qbits is f{min(qbits_dict.keys())}.\nChoosing k = 7')
#        k = 7.
    
    # find budget, best allocation, results
    if len(etf) == 0: 
        B = copy(budget)
        mdl, grouping = qcmodel(prices, k, B, mu, Q)
        optim_dict["docplex_mod"] = mdl
        results = aggregator('optimizer', optim_dict)
    
        # Integer results (amount of groups of stocks): x
        x_val = [results['result'].variables_dict[f'x{i}'] for i in range(len(prices))]
    
        # Amount of individual stock
        best_allocation = grouping*np.array(x_val)
        budget_spent = np.sum(best_allocation * prices) 
    else: 
        previous_month_etf = copy(etf[previous_month(date_).strftime('%Y-%m-%d')])
        B = previous_month_etf['liquidity'] + np.sum(np.array(previous_month_etf['allocation']) * prices)
        
        mdl, grouping = qcmodel(prices, k, B, mu, Q)
        optim_dict["docplex_mod"] = mdl
        results = aggregator('optimizer', optim_dict)
    
        # Integer results (amount of groups of stocks): x
        x_val = [results['result'].variables_dict[f'x{i}'] for i in range(len(prices))]
    
        # Amount of individual stock
        best_allocation = grouping*np.array(x_val)
        budget_spent = np.sum(best_allocation * prices)
        
    #printing tmp_results
    tmp_results = {
        'computational_time': float(results['computational_time']),
        'optimal_function_value': float(results['result'].fval), 
        'status': str(results['result'].status).split('.')[-1]
    }
    
    # generate etf datapoint
    etf[date_.strftime('%Y-%m-%d')] = {
        'allocation': [int(i) for i in best_allocation],
        'prices': [float(p) for p in prices],
        'liquidity': float(B - budget_spent),
        'portfolio_value': float(budget_spent),
        'results': copy(tmp_results)
        }
    with open('etf_results.json', 'wt') as json_results:
        json.dump(etf, json_results)
       
    # next datapoint
    date_ = next_month(date_)
    
    # clearing notebook output
    clear_output()  